In [1]:
import os
import glob
import albumentations as A
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [2]:
class myCustomdataset(Dataset):
    def __init__(self, img_path, transform):
        self.img_paths = glob.glob(os.path.join(img_path,'*'))
        self.transform = transform
    
    def __getitem__(self, index):
        img_path = self.img_paths[index]
        img = Image.open(img_path).convert('L')
        #img = transforms.ToTensor()(img)
        #img = transforms.Resize((28,28))(img)
        
        label = img_path.split('_')[2]
        label = int(label.split('.')[0])
        
        if self.transform:
            img = self.transform(img)
            
        return img, label
    
    def __len__(self):
        return len(self.img_paths)

In [3]:
transform = transforms.Compose([
    transforms.Resize((28,28)),
    transforms.ToTensor()
])

data_set = myCustomdataset('./data/hand_written/', transform=transform)

test_len = int(len(data_set)*0.2)
train_len = len(data_set) - test_len

train_subset, test_subset = random_split(data_set, [train_len, test_len]) # (나눠야할 데이터 셋, 각각 나눌 갯수(정수 갯수))
train_dataset = train_subset.dataset
test_dataset = test_subset.dataset

train_loader = DataLoader(train_dataset, 32, shuffle=True)
test_loader = DataLoader(test_dataset, 32, shuffle=True)

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(32*7*7, 10)
    
    def forward(self,x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

In [6]:
for epoch in range(num_epochs):
    running_loss = 0.0
    for imgs, labels in train_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)
        
        output = model(imgs)
        loss = criterion(output,labels).to(device)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * imgs.size(0)
    
    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch: {epoch+1}, Loss: {epoch_loss}')

Epoch: 1, Loss: 70.70987141132355
Epoch: 2, Loss: 70.14913642406464
Epoch: 3, Loss: 69.96906065940857
Epoch: 4, Loss: 69.57421898841858
Epoch: 5, Loss: 68.82429587841034
Epoch: 6, Loss: 67.42434883117676
Epoch: 7, Loss: 65.44997024536133
Epoch: 8, Loss: 63.514851212501526
Epoch: 9, Loss: 60.60713994503021
Epoch: 10, Loss: 58.412759363651276


In [7]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)
        
        outputs = model(imgs)
        _, predicted = torch.max(outputs.data, 1)
        
        labels = [int(label) for label in labels]
        labels = torch.tensor(labels)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    accuracy = 100 * correct / total
    print(f'test accuracy: {accuracy:.2f}')

test accuracy: 42.01
